#### Mã hóa tin nhắn 
Quy trình Mã hóa tin nhắn:

1.Chọn khóa bí mật để mã hóa tin nhắn. Khóa này là mật khẩu mà chỉ người được cho là đọc tin nhắn mới biết.

2.Sau đó lấy tin nhắn và xáo trộn nó bằng phím. Có nghĩa là biến tin nhắn thành một mã bí mật giống như một mớ chữ cái hoặc con số lộn xộn.

3.Sau khi tin nhắn được mã hóa, nó sẽ được gửi đến người được yêu cầu đọc nó. Nhưng nếu người khác cố đọc nó, họ sẽ thấy một mớ chữ hoặc số lộn xộn vô nghĩa.
Vì vậy, Mã hóa tin nhắn giống như có một mã bí mật mà chỉ người được cho là đọc tin nhắn mới có thể hiểu được.

# pip install cryptography

In [1]:
#VD về 1 đoạn tin nhắn
message_data = {
    "Aman": [
        {"message": "Hey Divyansha, how's it going?", "time": "2023-03-21 10:30:00"},
        {"message": "Not too bad, just working on some coding projects. Did you hear about the new encryption algorithm?", "time": "2023-03-21 10:35:00"},
        {"message": "It's called AES256 and it's supposed to be really secure. Want to give it a try with our messages?", "time": "2023-03-21 10:40:00"},
    ],
    "Divyansha": [
        {"message": "Good, thanks! How about you?", "time": "2023-03-21 10:32:00"},
        {"message": "No, what's that?", "time": "2023-03-21 10:37:00"},
        {"message": "Sure, let's do it!", "time": "2023-03-21 10:42:00"},
    ]
}

In [2]:
import os #os là một module được tích hợp sẵn, cho phép chúng ta tương tác với hệ điều hành và thực hiện các thao tác liên quan đến tệp và thư mục

#import các module cần thiết từ thư viện cryptography để thực hiện việc mã hóa và giải mã dữ liệu.
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
#default_backend để sử dụng backend mặc định của cryptography
from cryptography.hazmat.backends import default_backend
#Tạo một khóa bí mật ngẫu nhiên có độ dài là 32 byte
shared_secret_key = os.urandom(32)

Xác định một chức năng để giữ an toàn cho tin nhắn bí mật khỏi những người không nên xem nó:

In [3]:
def encrypt_message(message, key):
    iv = os.urandom(16)   #Tạo một vector khởi tạo (initialization vector - iv) ngẫu nhiên có độ dài là 16 byte
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend()) #Tạo một đối tượng Cipher với thuật toán AES và chế độ CBC, sử dụng khóa và IV đã được cung cấp
    encryptor = cipher.encryptor()  #Tạo một encryptor từ đối tượng Cipher đã tạo, sẵn sàng để mã hóa dữ liệu.
    padded_message = message + (16 - len(message) % 16) * chr(16 - len(message) % 16)  #Đảm bảo rằng độ dài của thông điệp là bội số của 16 byte bằng cách thêm ký tự đệm. 
    ciphertext = encryptor.update(padded_message.encode()) + encryptor.finalize()  #Mã hóa thông điệp đã được đệm bằng cách sử dụng encryptor, sau đó kết hợp với phần dữ liệu cuối cùng (finalization).
    return iv + ciphertext   #Trả về IV kèm với dữ liệu đã được mã hóa.

Xác định một hàm để giải mã thông điệp bí mật được tạo bằng khóa và mã hóa:

In [4]:
def decrypt_message(ciphertext, key):
    iv = ciphertext[:16]  #Trích xuất vector khởi tạo (IV) từ ciphertext. IV là 16 byte đầu tiên của ciphertext.
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())  #Tạo một đối tượng Cipher với thuật toán AES và chế độ CBC, sử dụng khóa và IV đã được trích xuất.
    decryptor = cipher.decryptor()  #Tạo một decryptor từ đối tượng Cipher, sẵn sàng để giải mã dữ liệu.
    plaintext = decryptor.update(ciphertext[16:]) + decryptor.finalize()  #Giải mã dữ liệu đã được mã hóa, bỏ qua phần IV và sau đó kết hợp với phần dữ liệu cuối cùng (finalization).
    padding_length = plaintext[-1]   #Xác định độ dài của ký tự đệm được thêm vào cuối thông điệp đã giải mã.
    plaintext = plaintext[:-padding_length]   #Loại bỏ ký tự đệm từ cuối của thông điệp đã giải mã.
    return plaintext.decode()   #Trả về thông điệp đã giải mã dưới dạng chuỗi Unicode.

Cách mã hóa từ điển bằng các tin nhắn được mã hóa để chỉ những người biết khóa mới có thể giải mã và đọc các tin nhắn:

In [5]:
for person, messages in message_data.items(): #
    for message in messages:
        encrypted_message = encrypt_message(message["message"], shared_secret_key)  #mã hóa nội dung của tin nhắn bằng cách sử dụng khóa chia sẻ
        message["message"] = encrypted_message.hex()  #Chuyển đổi kết quả mã hóa thành chuỗi hex và ghi đè lại vào, thay thế nội dung tin nhắn gốc bằng nội dung đã mã hóa dưới dạng chuỗi hex.

print("Encrypted message_data dictionary:")
print(message_data)

Encrypted message_data dictionary:
{'Aman': [{'message': 'a50428061cb125a947b297271c66c1a1c4369b5525025f0350ffa68a281e9c8163a7a6bb1c0293de00dea2cecd7ef3e9', 'time': '2023-03-21 10:30:00'}, {'message': '98b8073797147f703efb8dddd5500605d28717157f19f6ae557f1b120fa3a084040b23528b40f9cd71876ea35bb142aead28a5419c00fb5ed245ae94a4aabb95be10646c7aaa61feaa3fba5130abd3b8657a6024364121871fb1e718974612ea55ca24195ef2b98b5ba9e99fe3b9910c36e55e25b7012615e9fbc299eaa03ade', 'time': '2023-03-21 10:35:00'}, {'message': '71484c2e202347cd8ff297d174dc7a94f8550d185086649279132a98ad460b9b7a37f48995e8cf8f406d62419fb82fcad95cdc7d73df6f612eafa4e993ca8d6b15abe170c7e756c747bbcaafed431242154d5cf42013fe3e5d681f7d1cb2168a960d20c004660c89e05797839ac84a187088d4b051bbb78d1013cd3427e8b73a', 'time': '2023-03-21 10:40:00'}], 'Divyansha': [{'message': '5ec81b14998c8e7da23c15a0dd984d41b9727ac60bcb14a46da71046dcc664e856002bc55716c1dd590f99870c91291e', 'time': '2023-03-21 10:32:00'}, {'message': '476c2d36d1e933784e0ea9dd121ebe6

Cách giải mã các tin nhắn được mã hóa để những người có chìa khóa có thể đọc được tin nhắn:

In [6]:
for person, messages in message_data.items():
    for message in messages:
        ciphertext = bytes.fromhex(message["message"])  #Chuyển đổi chuỗi hex của tin nhắn đã được mã hóa thành dữ liệu byte 
        decrypted_message = decrypt_message(ciphertext, shared_secret_key)  #giải mã dữ liệu đã được mã hóa bằng cách sử dụng khóa chia sẻ.
        message["message"] = decrypted_message   #Ghi đè lại nội dung của tin nhắn bằng nội dung đã được giải mã.

print("Decrypted message_data dictionary:")
print(message_data)

Decrypted message_data dictionary:
{'Aman': [{'message': "Hey Divyansha, how's it going?", 'time': '2023-03-21 10:30:00'}, {'message': 'Not too bad, just working on some coding projects. Did you hear about the new encryption algorithm?', 'time': '2023-03-21 10:35:00'}, {'message': "It's called AES256 and it's supposed to be really secure. Want to give it a try with our messages?", 'time': '2023-03-21 10:40:00'}], 'Divyansha': [{'message': 'Good, thanks! How about you?', 'time': '2023-03-21 10:32:00'}, {'message': "No, what's that?", 'time': '2023-03-21 10:37:00'}, {'message': "Sure, let's do it!", 'time': '2023-03-21 10:42:00'}]}
